In [1]:
%load_ext autoreload
%autoreload 2 

In [146]:
import numpy as np
import pandas as pd
import re 
import pytest
import sys
import re
np.set_printoptions(suppress=True)
pd.options.display.float_format = "{:,.2f}".format

In [3]:
import best_subset as bs_pkgs


In [164]:

def process_exaustive_and_return_top(df, features, return_top=100):
    mask = df['Models'].notna()  # Ensure we exclude NaN values    
    for feature in features:
        mask &= df['Models'].str.contains(rf'\b{feature}\b', regex=True, na=False)
    top =  df[mask] 

    top['rank'] = top.groupby("Var Number")['Scores'].rank(ascending=False)
    top = top[top['rank']<=100]
    top.drop('rank', axis=1, inplace=True)
    top = top.reset_index(drop=True)
    return top


def check_if_features_in(df, features):
    mask = df['Models'].notna()  # Ensure we exclude NaN values    
    for feature in features:
        mask &= df['Models'].str.contains(rf'\b{feature}\b', regex=True, na=False)
    return df[mask]

def compare_dataframes(df1: pd.DataFrame, df2: pd.DataFrame):
    """Compares two pandas DataFrames, rounding floating-point columns to 2 decimal places."""
    float_cols = df1.select_dtypes(include=['float']).columns
    df1_rounded = df1.copy()
    df2_rounded = df2.copy()
    df1_rounded[float_cols] = df1_rounded[float_cols].round(2)
    df2_rounded[float_cols] = df2_rounded[float_cols].round(2)
    pd.testing.assert_frame_equal(df1_rounded, df2_rounded, check_dtype=False)
    print("Dataset Match")

def order_models_field(df):
    df['Models'] = df['Models'].apply(
        lambda model: " ".join(
            sorted(model.split(" "), key=lambda x: int(re.search(r'\d+', x).group()))
        )
    )
    df = df.reset_index(drop=True)
    return df 

def order_models_filed_all(df):
    df['Models'] = df['Models'].apply(lambda model: " ".join(sorted(model.split(" "))) )
    return df


def create_synthetic_data(seed=42, n=50000, p=15):
    """
    Creates a DataFrame X of shape (n, p+1) with columns:
      - 'const': all ones (intercept)
      - 'x1', 'x2', ... 'x15'
    And a Series y with binary (0/1) outcomes drawn from a logistic model.
    
    Some of the 15 features have nonzero coefficients, others are zero,
    so there's meaningful signal to detect in a logistic regression.
    """

    np.random.seed(seed)

    # 1) Generate random features ~ N(0,1)
    X_base = np.random.randn(n, p)
    
    # 2) Define "true" coefficients
    #    For instance, let's say 5 features matter:
    #    x1, x2, x3, x4, x5 have some nonzero betas.
    #    The remaining x6..x15 have 0 effect.
    betas_true = np.array([1.5, -2.0, 0.75, 1.25, -0.5] + [0]*(p-5))
    #     -> length = 15
    
    # 3) Linear predictor: X_base dot betas_true
    #    shape -> (n, )
    lin_pred = X_base.dot(betas_true)
    
    # 4) Convert linear predictor to probability via logistic function
    #    p_i = 1 / (1 + exp(-lin_pred))
    prob = 1.0 / (1.0 + np.exp(-lin_pred))
    
    # 5) Draw binary outcomes y from Bernoulli(prob)
    y_vals = np.random.binomial(1, prob)
    
    # 6) Create a DataFrame for features, plus an intercept column
    df = pd.DataFrame(X_base, columns=[f"x{i+1}" for i in range(p)])
    df.insert(0, "const", 1.0)  # intercept
    
    # 7) Create a Series for y
    y = pd.Series(y_vals, name="y")
    
    return df, y


def create_synthetic_data_weights(seed=42, n=50000, p=15):
    """
    Creates a DataFrame X of shape (n, p+2) with columns:
      - 'const': all ones (intercept)
      - 'x1', 'x2', ... 'x15'
      - 'weight': randomly generated weights between 0 and 100
    And a Series y with binary (0/1) outcomes drawn from a logistic model.
    
    Some of the 15 features have nonzero coefficients, others are zero,
    so there's meaningful signal to detect in a logistic regression.
    """

    np.random.seed(seed)

    # 1) Generate random features ~ N(0,1)
    X_base = np.random.randn(n, p)
    
    # 2) Define "true" coefficients
    #    For instance, let's say 5 features matter:
    #    x1, x2, x3, x4, x5 have some nonzero betas.
    #    The remaining x6..x15 have 0 effect.
    betas_true = np.array([1.5, -2.0, 0.75, 1.25, -0.5] + [0]*(p-5))
    #     -> length = 15
    
    # 3) Linear predictor: X_base dot betas_true
    #    shape -> (n, )
    lin_pred = X_base.dot(betas_true)
    
    # 4) Convert linear predictor to probability via logistic function
    #    p_i = 1 / (1 + exp(-lin_pred))
    prob = 1.0 / (1.0 + np.exp(-lin_pred))
    
    # 5) Draw binary outcomes y from Bernoulli(prob)
    y_vals = np.random.binomial(1, prob)
    
    # 6) Create a DataFrame for features, plus an intercept column
    df = pd.DataFrame(X_base, columns=[f"x{i+1}" for i in range(p)])
    df.insert(0, "const", 1.0)  # intercept

    # 7) Generate weights between 0 and 100
    weights = np.random.uniform(0, 100, size=n)
    df['weight'] = weights  # Add 'weight' column
    
    # 8) Create a Series for y
    y = pd.Series(y_vals, name="y")
    
    return df, y


# Example usage
if __name__ == "__main__":
    df, y = create_synthetic_data_weights(p=15)
    # print(df.head())
    # print(y.head())
    print(df.columns)
    print("df shape:", df.shape)
    print("y shape:", y.shape)

Index(['const', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10',
       'x11', 'x12', 'x13', 'x14', 'x15', 'weight'],
      dtype='object')
df shape: (50000, 17)
y shape: (50000,)


In [125]:
candidates

['x1',
 'x2',
 'fico',
 'fico_lt',
 'gt_fico',
 'x6',
 'x7',
 'x8',
 'x9',
 'x10',
 'x11',
 'x12',
 'x13',
 'x14',
 'x15']

In [114]:

df, y = create_synthetic_data_weights(p=15)
df.rename(columns ={ "x3": 'fico', "x4": 'fico_lt', "x5": "fico_gt",  "x5": "gt_fico"}, inplace=True)
candidates = df.columns[1:-1].tolist()
results =  bs_pkgs.best_subset_exhaustive_logistic(df, y, candidates)


Finished Var Family: 1
Finished Var Family: 2
Finished Var Family: 3
Finished Var Family: 4
Finished Var Family: 5
Finished Var Family: 6
Finished Var Family: 7
Finished Var Family: 8
Finished Var Family: 9
Finished Var Family: 10
Finished Var Family: 11
Finished Var Family: 12
Finished Var Family: 13
Finished Var Family: 14
Finished Var Family: 15
Total Models: 32767


In [135]:
# "*fico*".replace("*", "")

'fico'

In [165]:

def check_if_features_in(df, features):
    mask = df['Models'].notna()  # Ensure we exclude NaN values    
    for feature in features:
        if "*" in feature:
            feature = feature.replace("*", "")
            mask &= df['Models'].str.contains(f'{feature}', case=False)
        else:        
            mask &= df['Models'].str.contains(rf'\b{feature}\b', regex=True, na=False)
    return df[mask]
# print(a_bfs_v2.shape, check_if_features_in(a_bfs_v2, features).shape)
top = check_if_features_in(results, ['x1','x2', '*fico*'])
top = order_models_filed_all(top)
top['rank'] = top.groupby("Var Number")['Scores'].rank(ascending=False)
top = top[top['rank']<=100]
top.drop('rank', axis=1, inplace=True)
top = top.reset_index(drop=True)

C:\Users\enisb\AppData\Local\Temp\ipykernel_115864\2013603733.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Models'] = df['Models'].apply(lambda model: " ".join(sorted(model.split(" "))) )
C:\Users\enisb\AppData\Local\Temp\ipykernel_115864\4003759345.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top['rank'] = top.groupby("Var Number")['Scores'].rank(ascending=False)


In [166]:
# pd.Series([True, True])  & pd.Series([True, False])
# top = check_if_features_in(results, ['x1','x2', 'fico'])
top

,Var Number,Models,Scores
0,3,fico_lt x1 x2,"21,162.88"
1,3,fico x1 x2,"18,375.78"
2,3,gt_fico x1 x2,"17,480.30"
3,4,fico fico_lt x1 x2,"22,728.72"
4,4,fico_lt gt_fico x1 x2,"21,831.46"
...,...,...,...
923,14,fico fico_lt gt_fico x1 x10 x11 x13 x14 x15 x2...,"23,399.42"
924,14,fico fico_lt x1 x10 x11 x12 x13 x14 x15 x2 x6 ...,"22,735.39"
925,14,fico_lt gt_fico x1 x10 x11 x12 x13 x14 x15 x2 ...,"21,838.16"
926,14,fico gt_fico x1 x10 x11 x12 x13 x14 x15 x2 x6 ...,"19,051.80"


In [126]:
results[results['Models'].str.contains(r'\bfico\b', regex=True, na=False)].query("`Var Number`==3")

,Var Number,Models,Scores
120,3,x1 x2 fico,"18,375.78"
211,3,x2 fico fico_lt,"16,660.41"
212,3,x2 fico gt_fico,"12,917.91"
219,3,x2 fico x12,"12,282.14"
213,3,x2 fico x6,"12,279.18"
...,...,...,...
328,3,fico x8 x10,"1,628.96"
331,3,fico x8 x13,"1,628.96"
340,3,fico x10 x11,"1,628.94"
346,3,fico x11 x13,"1,628.94"


In [119]:
top
# candidates

,Var Number,Models,Scores
0,3,x1 x2 fico,"18,375.78"
1,4,x1 x2 fico fico_lt,"22,728.72"
2,4,x1 x2 fico gt_fico,"19,045.94"
3,4,x1 x2 fico x12,"18,379.35"
4,4,x1 x2 fico x15,"18,376.91"
...,...,...,...
853,14,x1 x2 fico fico_lt gt_fico x7 x8 x9 x10 x11 x1...,"23,400.77"
854,14,x1 x2 fico fico_lt gt_fico x6 x7 x8 x9 x10 x11...,"23,399.42"
855,14,x1 x2 fico fico_lt x6 x7 x8 x9 x10 x11 x12 x13...,"22,735.39"
856,14,x1 x2 fico gt_fico x6 x7 x8 x9 x10 x11 x12 x13...,"19,051.80"


In [107]:
%%time
df, y = create_synthetic_data_weights(p=15)
df.rename(columns ={ "x3": 'fico', "x4": 'fico_lt', "x5": "fico_gt",  "x5": "gt_fico"}, inplace=True)

cands = df.columns[1:].tolist()

cands.remove('weight')
print(cands)
res_noweights, b_bfs_v2, c_bfs_v2= bs_pkgs.best_subset_bb_logistic_weighted(df, y, 100, start=2, stop=10,  candidates=cands ,  forced_vars=['x1', 'x2'])
res_noweights.shape

['x1', 'x2', 'fico', 'fico_lt', 'gt_fico', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15']
Finished Var Family: 2
Finished Var Family: 3
Finished Var Family: 4
Finished Var Family: 5
Finished Var Family: 6
Finished Var Family: 7
Finished Var Family: 8
Finished Var Family: 9
Finished Var Family: 10
100
18
CPU times: total: 641 ms
Wall time: 204 ms


(692, 3)

In [167]:
%%time
df, y = create_synthetic_data_weights(p=15)
df.rename(columns ={ "x3": 'fico', "x4": 'fico_lt', "x5": "fico_gt",  "x5": "gt_fico"}, inplace=True)
cands = df.columns[1:].tolist()
cands.remove('weight')
print(cands)
res_weights, b_bfs_v2, c_bfs_v2= bs_pkgs.best_subset_bb_logistic_weighted_started(df, y, 100, start=2, stop=10,  candidates=cands,  forced_vars=['x1', 'x2']  )
# res_weights, b_bfs_v2, c_bfs_v2= bs_pkgs.best_subset_bb_logistic_weighted(df, y, 100, start=2, stop=10,  candidates=cands  )
res_weights.shape

['x1', 'x2', 'fico', 'fico_lt', 'gt_fico', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15']
Finished Var Family: 2  Skipped
Finished Var Family: 3
Finished Var Family: 4
Finished Var Family: 5
Finished Var Family: 6
Finished Var Family: 7
Finished Var Family: 8
Finished Var Family: 9
Finished Var Family: 10
100
18
CPU times: total: 844 ms
Wall time: 315 ms


(636, 3)

In [108]:
compare_dataframes(res_noweights,res_weights)
# compare_dataframes(top,res_weights)


Dataset Match


In [171]:
%%time
df, y = create_synthetic_data_weights(p=15)
df.rename(columns ={ "x3": 'fico', "x4": 'fico_lt', "x5": "fico_gt",  "x5": "gt_fico"}, inplace=True)
cands = df.columns[1:].tolist()
cands.remove('weight')
print(cands)
res_weights, b_bfs_v2, c_bfs_v2= bs_pkgs.best_subset_bb_logistic_weighted_started(df, y, 100, start=2, stop=15,  candidates=cands,  forced_vars=['x1',  'x2',    "*fico*"])
res_weights = order_models_filed_all(res_weights)
res_weights = res_weights.reset_index(drop=True)
res_weights

['x1', 'x2', 'fico', 'fico_lt', 'gt_fico', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15']
Finished Var Family: 2  Skipped
Finished Var Family: 3
Finished Var Family: 4
Finished Var Family: 5
Finished Var Family: 6
Finished Var Family: 7
Finished Var Family: 8
Finished Var Family: 9
Finished Var Family: 10
Finished Var Family: 11
Finished Var Family: 12
Finished Var Family: 13
Finished Var Family: 14
Finished Var Family: 15
100
29
CPU times: total: 922 ms
Wall time: 401 ms


,Var Number,Models,Scores
0,3,fico_lt x1 x2,"21,162.88"
1,3,fico x1 x2,"18,375.78"
2,3,gt_fico x1 x2,"17,480.30"
3,4,fico fico_lt x1 x2,"22,728.72"
4,4,fico_lt gt_fico x1 x2,"21,831.46"
...,...,...,...
923,14,fico fico_lt gt_fico x1 x10 x11 x13 x14 x15 x2...,"23,399.42"
924,14,fico fico_lt x1 x10 x11 x12 x13 x14 x15 x2 x6 ...,"22,735.39"
925,14,fico_lt gt_fico x1 x10 x11 x12 x13 x14 x15 x2 ...,"21,838.16"
926,14,fico gt_fico x1 x10 x11 x12 x13 x14 x15 x2 x6 ...,"19,051.80"


In [172]:
compare_dataframes(top,res_weights)


Dataset Match


In [125]:
def loglike(params, X, y, W):
    q = 2 * y - 1
    return np.sum(W * np.log(cdf(q * np.dot(X, params))))

def cdf(X):
    X = np.asarray(X)
    return 1 / (1 + np.exp(-X))

def I(params, X, W):
    X = np.array(X)
 
    L = cdf(np.dot(X, params))
    return -np.dot(W * L * (1 - L) * X.T, X)

def score(params, X, y, W):    
    L = cdf(np.dot(X, params))
    return np.dot(W * (y - L), X)

cands = ['const', 'x1', 'x2']
df[cands]
X = np.asarray(df[cands])
weights = np.asarray(df['weight'])
nobs = df.shape[0]
W = (weights / np.sum(weights)) * nobs # normalized weights
W = weights
avg = np.sum(W * y) / np.sum(W)
null_model = np.log(avg / (1 - avg))
theta_0 = np.append(null_model, np.zeros(X.shape[1] - 1))
g = score(theta_0, X, y, W)
Is = I(theta_0, X, W)
-g.T.dot(np.linalg.inv(Is)).dot(g)

np.float64(845526.7225987429)

In [112]:
isinstance(df['weight'].values, np.ndarray)

True

In [203]:
%%time
a_5, b_5, c_5= bs6.best_subset_bb_logistic_with_priority(df, y, 100, start=2, stop=5,  method="dfs", candidates=df.columns[1:].tolist()) # , forced_vars=['x1', 'x2'])
a_5

Finished Var Family: 2
Finished Var Family: 3
Finished Var Family: 4
Finished Var Family: 5
CPU times: total: 2min 50s
Wall time: 42.3 s


,Var Number,Models,Scores
99,2,x2 x1,17140.282747
98,2,x2 x4,15300.586378
97,2,x2 x3,12622.364252
96,2,x2 x5,11698.923275
95,2,x2 x48,11036.236241
...,...,...,...
304,5,x2 x1 x4 x18 x15,21418.674221
303,5,x2 x1 x4 x15 x45,21418.662492
302,5,x2 x1 x4 x13 x48,21418.614501
301,5,x2 x1 x4 x15 x37,21418.472611


In [214]:
%%time
a_5, b_5, c_5= bs6.best_subset_bb_logistic_with_priority(df, y, 100, start=2, stop=12,   method="best_first",  candidates=df.columns[1:].tolist()) # , forced_vars=['x1', 'x2'])


Finished Var Family: 2
Finished Var Family: 3
Finished Var Family: 4
Finished Var Family: 5
Finished Var Family: 6
Finished Var Family: 7
Finished Var Family: 8
Finished Var Family: 9
Finished Var Family: 10
Finished Var Family: 11
Finished Var Family: 12
CPU times: total: 3min 20s
Wall time: 53.4 s


In [217]:
%%time
a_5, b_5, c_5= bs7.best_subset_bb_logistic_with_priority(df, y, 100, start=2, stop=12,  method="best_first", candidates=df.columns[1:].tolist()) # , forced_vars=['x1', 'x2'])


Finished Var Family: 2
Finished Var Family: 3
Finished Var Family: 4
Finished Var Family: 5
Finished Var Family: 6
Finished Var Family: 7
Finished Var Family: 8
Finished Var Family: 9
Finished Var Family: 10
Finished Var Family: 11
Finished Var Family: 12
CPU times: total: 24 s
Wall time: 29.1 s


In [216]:
b_5

302568

In [184]:
%%time
a,b = bs3.best_subset_bb_logistic(df, y, 100, start=2, stop=5, candidates=df.columns[1:].tolist()) # , forced_vars=['x1', 'x2'])
a

Finished Var Family: 2
Finished Var Family: 3
Finished Var Family: 4
Finished Var Family: 5
CPU times: total: 19min 18s
Wall time: 5min 10s


,Var Number,Models,Scores
99,2,x1 x2,17140.282747
98,2,x2 x4,15300.586378
97,2,x2 x3,12622.364252
96,2,x2 x5,11698.923275
95,2,x2 x48,11036.236241
...,...,...,...
304,5,x1 x2 x4 x15 x18,21418.674221
303,5,x1 x2 x4 x15 x45,21418.662492
302,5,x1 x2 x4 x13 x48,21418.614501
301,5,x1 x2 x4 x15 x37,21418.472611


In [185]:
%%time
a,b = bs4.best_subset_bb_logistic(df, y, 100, start=2, stop=5, candidates=df.columns[1:].tolist()) # , forced_vars=['x1', 'x2'])
a

CPU times: total: 0 ns
Wall time: 0 ns


In [163]:
%%time
 
a_par,b = bs5.best_subset_bb_logistic_parallel(df, y, nbest=100, start=2, stop=15,  candidates=df.columns[1:].tolist()) # , forced_vars=['x1', 'x2'])

Finished Var Size = 2, best_bound = 23402.040
Finished Var Size = 3, best_bound = 23402.040
Finished Var Size = 4, best_bound = 23402.040
Finished Var Size = 5, best_bound = 23402.040
Finished Var Size = 6, best_bound = 23402.040
Finished Var Size = 7, best_bound = 23402.040
Finished Var Size = 8, best_bound = 23402.040
Finished Var Size = 9, best_bound = 23402.040
Finished Var Size = 10, best_bound = 23402.040
Finished Var Size = 11, best_bound = 23402.040
Finished Var Size = 12, best_bound = 23402.040
Finished Var Size = 13, best_bound = 23402.040
Finished Var Size = 14, best_bound = 23402.040
Finished Var Size = 15, best_bound = 23402.040
Total expansions: 224
CPU times: total: 609 ms
Wall time: 484 ms


In [205]:
import numpy as np
import scipy.linalg
import time

# Generate a random symmetric positive-definite matrix I and a random vector v
np.random.seed(42)
n = 500  # Matrix size
I = np.random.rand(n, n)
I = np.dot(I, I.T)  # Make it symmetric positive-definite
v = np.random.rand(n)

# Method 1: Using np.linalg.inv
start_inv = time.time()
I_inv = np.linalg.inv(I)
result_inv = v.T @ I_inv @ v
end_inv = time.time()

# Method 2: Using scipy.linalg.cho_solve
start_cho = time.time()
L = scipy.linalg.cho_factor(I)  # Perform Cholesky factorization
result_cho = v.T @ scipy.linalg.cho_solve(L, v)  # Solve using the factorization
end_cho = time.time()

# Method 3: Using np.linalg.solve
start_solve = time.time()
result_solve = v.T @ np.linalg.solve(I, v)  # Directly solve the system
end_solve = time.time()

# Print results
print("Results:")
print(f"Result using np.linalg.inv: {result_inv}")
print(f"Result using scipy.linalg.cho_solve: {result_cho}")
print(f"Result using np.linalg.solve: {result_solve}")

print("\nTiming:")
print(f"Time using np.linalg.inv: {end_inv - start_inv:.6f} seconds")
print(f"Time using scipy.linalg.cho_solve: {end_cho - start_cho:.6f} seconds")
print(f"Time using np.linalg.solve: {end_solve - start_solve:.6f} seconds")
 

Results:
Result using np.linalg.inv: 395.2539871247952
Result using scipy.linalg.cho_solve: 395.2539871187649
Result using np.linalg.solve: 395.2539871247954

Timing:
Time using np.linalg.inv: 0.019066 seconds
Time using scipy.linalg.cho_solve: 0.011001 seconds
Time using np.linalg.solve: 0.003004 seconds


In [219]:
import heapq

# Create a heap
heap = []
heapq.heappush(heap, 10)
heapq.heappush(heap, 5)
heapq.heappush(heap, 20)
type(heap)

list

In [220]:
 


class Node:
    def __init__(self, key, branches, n, forced_vars=None):
        """
        key: the current subset of variables (excluding 'const')
        branches: how many branches remain
        n: the target subset size or node parameter
        forced_vars: list of variables that must stay in every subset
        """
        if forced_vars is None:
            forced_vars = []

        self.key = key                # full subset (list of strings)
        self.key2 = key[:n]           # partial subset for bounding
        self.branch_id = n - branches + 1
        self.n = n
        self.forced_vars = forced_vars

        self.child = []
        self.key_list = []
        self.has_branches = branches

    def add_children(self):
        """
        Create child nodes by popping one feature at a time
        but skip if that would drop any forced_var from the subset.
        """
        visit = self.has_branches - 1

        for has_branches_new, _ in enumerate(range(visit, 0, -1)):
            child_branch_id = self.n - has_branches_new - 1
            temp = self.key[:]

            # Sanity check: child_branch_id might be out of range
            if child_branch_id < 0 or child_branch_id >= len(temp):
                continue

            removed_feat = temp.pop(child_branch_id)
            # If removing that feature leads to losing forced var, skip
            if removed_feat in self.forced_vars:
                continue

            # Also skip if after removal, any forced var is not in temp
            if not all(fv in temp for fv in self.forced_vars):
                continue

            # If we haven't pruned, then child is valid
            # We also skip if it doesn't actually reduce the subset size
            if len(temp) == self.n - 1:
                # This line in the original code was used to skip 
                # same-size sets, but let's keep it for consistency:
                continue

            new_node = Node(
                temp, 
                has_branches_new + 2, 
                self.n, 
                forced_vars=self.forced_vars
            )
            self.child.append(new_node)
            self.key_list.append(temp)

In [221]:
var = 2
n = var 
root = Node(['x1', 'x2'] , branches=n + 1, n=n, forced_vars=None)

In [224]:
pq = []
heapq.heappush(pq, (-250, root))

In [225]:
pq

[(-250, <__main__.Node at 0x1a544ce6b60>)]